In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance

In [ ]:
IBMQ.load_account()

In [ ]:
def test_addition_modulo_p(a,b,p,n,g_s):
    x = QuantumRegister(n,name='x')
    y = QuantumRegister(n,name='y')
    c = QuantumRegister(1,name='d')
    g = QuantumRegister(1,name='g')
    clas_a = ClassicalRegister(n)
    clas_b = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    clas_g = ClassicalRegister(1)
    circuit = QuantumCircuit(x,y,c,g,clas_a,clas_b,clas_c,clas_g)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_p=s_p[::-1]
    s_p1=s_p[::]+'0'
    if(g_s=='1'):
        circuit.x(g)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])
    ECC_library.addition(circuit,x,y,c,n)
    ECC_library.constant_subtraction(circuit,y[0:n]+c[0:1],g,s_p1,n+1)
    ECC_library.controlled_constant_addition(circuit,c,y,g,s_p,n)
    ECC_library.comparator(circuit,y,x,c,n)
    circuit.x(c)
    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    for i in range(0, n):
        circuit.measure(y[i],clas_b[i])
    circuit.measure(c,clas_c)
    circuit.measure(g,clas_g)
    circuit.qasm()
    print('a:',a,'b:',b,'p:',p,'g:',g_s)
    provider = IBMQ.get_provider('ibm-q')
    backend = provider.get_backend('ibmq_qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_a = output[3]
    output_b = output[2]
    output_c = output[1]
    res_g = output[0]
    res_a = int(output_a, 2)
    res_b = int(output_b, 2)
    res_c = int(output_c, 2)
    print('res_a:',res_a,'res_b:',res_b,'res_c:',res_c,'res_g:',res_g)
    if (res_b!=(a+b)%p or res_c!=0 or res_g!=g_s or res_a!=a):
        print('found!')

In [ ]:
for n in range (4,6):
    for p in range (8,2**n):
        for i in range (0,p):
            for j in range (0,p):
                test_addition_modulo_p(i,j,p,n,'1')
                print('-----------')